In [1]:
import pandas as pd
from anthropic import Anthropic
from sklearn.metrics import matthews_corrcoef, accuracy_score, f1_score
from datasets import load_dataset

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test = pd.read_csv('../data/polnli_test_results.csv')

In [6]:
def metrics(df, preds, group_by=None):
    true_col = 'entailment'
    
    def get_metrics(y_true, y_pred):
        return {
            'MCC': matthews_corrcoef(y_true, y_pred),
            'Accuracy': accuracy_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred, average='weighted')
        }
    
    results = []
    
    if group_by not in ['dataset', 'task']:
        for col in preds:
            metrics = get_metrics(df[true_col], df[col])
            metrics['Column'] = col
            results.append(metrics)
    else:
        for col in preds:
            for group_name, group in df.groupby(group_by):
                metrics = get_metrics(group[true_col], group[col])
                metrics['Column'] = col
                metrics[group_by.capitalize()] = group_name
                results.append(metrics)
    
    results_df = pd.DataFrame(results)
    
    if group_by in ['dataset', 'task']:
        return results_df.set_index(['Column', group_by.capitalize()])
    else:
        return results_df.set_index('Column')

In [4]:
claude = Anthropic(api_key="##################")

In [5]:
user_message = """You are a classifier that can only respond with 0 or 1. I'm going to show you a short text sample and I want you to determine if {hypothesis}. Here is the text:
{doc}

If it is true that {hypothesis}, return 0. If it is not true that {hypothesis}, return 1.
Do not explain your answer, and only return 0 or 1.
"""

In [6]:
%%time
user_message = user_message
model = "claude-3-5-sonnet-20240620"
data = test
labels = []

for i in data.index:
    doc = data.loc[i, 'premise']
    hypothesis = data.loc[i, 'augmented_hypothesis']
    res = claude.messages.create(
        max_tokens=2,
        messages=[
            {
                "role": "user",
                "content": user_message.format(doc = doc, hypothesis = hypothesis),
            }
        ],
        model=model,
        temperature = 0
    )
    labels.append(res.content[0].text)

CPU times: total: 35.2 s
Wall time: 4h 8min 42s


In [7]:
test['sonnet'] = labels
test['sonnet'] = test['sonnet'].replace({'1':1, '0':0, '2':-1})
test.to_csv('polnli_test_results.csv', index = False)

C:\Users\mikeb\AppData\Local\Temp\ipykernel_12576\1404300228.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['sonnet'] = test['sonnet'].replace({'1':1, '0':0, '2':-1})


In [7]:
metrics(test, preds = ['base_polnli', 'large_polnli', 'llama', 'sonnet'], group_by = None)

,MCC,Accuracy,F1
Column,,,
base_polnli,0.894269,0.948978,0.948852
large_polnli,0.915911,0.959326,0.959180
llama,0.730997,0.862358,0.863467
sonnet,0.815902,0.910517,0.909423


In [8]:
metrics(test, preds = ['base_polnli', 'large_polnli', 'llama', 'sonnet'], group_by = 'task')

MCC  Accuracy        F1
Column       Task                                                 
base_polnli  event extraction         0.813742  0.906774  0.907042
             hatespeech and toxicity  0.841410  0.946036  0.945248
             stance detection         0.920600  0.961546  0.961488
             topic classification     0.926241  0.963834  0.963826
large_polnli event extraction         0.819699  0.909567  0.909838
             hatespeech and toxicity  0.881535  0.959360  0.959026
             stance detection         0.969009  0.984979  0.984972
             topic classification     0.924496  0.962503  0.962322
llama        event extraction         0.808244  0.905726  0.905480
             hatespeech and toxicity  0.559060  0.782145  0.799067
             stance detection         0.605609  0.798117  0.799651
             topic classification     0.918734  0.959396  0.959505
sonnet       event extraction         0.784838  0.880936  0.881063
             hatespeech and toxicity  0.571282  0.862425  0.853626
             stance detection         0.791883  0.899259  0.898415
             topic classification     0.946775  0.973819  0.973765